## This Script Contains the Following:
1. Importing Libraries and Files
2. Reshaping the Data
3. Creating Bayesian Optimizer
4. Building CNN Model

## 01. Importing Libraries and Files

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from keras.layers import Dense, Dropout
from keras import utils
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

/opt/anaconda3/envs/notebook/lib/python3.12/site-packages/keras/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [2]:
path = r'/Users/niravbariya/Documents/Data Science/CareerFoundry/Machine Learning Specialization/Achievement 1 & 2/02 Data Sets'

In [3]:
tf.random.set_seed(42)

In [4]:
weather_cleaned = pd.read_csv(os.path.join(path, 'Unsupervised', 'weather_cleaned.csv'), index_col=False)

In [5]:
weather_pred = pd.read_csv(os.path.join(path, 'Unsupervised', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'),index_col=False)

In [6]:
weather_cleaned.head(3)

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.0,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.0304,0.20,0.1,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9,1960-01-01
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.0,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.0292,0.61,0.3,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1,1960-01-02
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.0,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0.00,0.6,2.7,1.7,5.3,

In [10]:
weather_cleaned = weather_cleaned.drop(columns = 'DATE')

In [11]:
weather_cleaned.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.0304,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.0292,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0.00,0.6,2.7,1.7,5.3,6,0.92,1.0165,0.28,0.0

In [12]:
weather_pred = weather_pred.drop(columns = 'DATE')

In [13]:
predictions = {0: 'BASEL', 1: 'BELGRADE', 2: 'BUDAPEST', 3: 'DEBILT', 4: 'DUSSELDORF', 5: 'HEATHROW', 6: 'KASSEL', 7: 'LJUBLJANA',
               8: 'MAASTRICHT', 9: 'MADRID', 10: 'MUNCHENB', 11: 'OSLO', 12: 'SONNBLICK', 13: 'STOCKHOLM', 14: 'VALENTIA'}

## 02. Reshaping the Data

In [136]:
X = weather_cleaned
y = weather_pred

In [137]:
X = np.array(X.iloc[:])

In [138]:
X = X.reshape(-1,15,9)

In [139]:
X.shape, y.shape

((22950, 15, 9), (22950, 15))

In [140]:
# Get rid of one-hot encoding and supply the numerical value.
y = np.argmax(y, axis = 1)
print(y.shape)
y

(22950,)


array([0, 0, 0, ..., 0, 0, 0])

In [141]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

In [143]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18360, 15, 9) (18360,)
(4590, 15, 9) (4590,)


In [145]:
# Verifying y_train
y_train

array([0, 1, 0, ..., 9, 9, 0])

In [103]:
# Verifying type
type_of_target(y_train)

'multiclass'

## 03. Creating Bayesian Optimizer

In [104]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 #_count_classes(y_train) Same as number of stations

#Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [146]:
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
            
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
            
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
            
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
            
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
        
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [147]:
#Running Bayesian Optimization
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), #9
    'optimizer':(0,7), #7
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), #(10, 50), #
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}

# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/32
16/16 - 1s - 74ms/step - accuracy: 0.6147 - loss: 2.6981
Epoch 2/32
16/16 - 1s - 43ms/step - accuracy: 0.6452 - loss: 2.6999
Epoch 3/32
16/16 - 1s - 43ms/step - accuracy: 0.6452 - loss: 2.6964
Epoch 4/32
16/16 - 1s - 43ms/step - accuracy: 0.6452 - loss: 2.6934
Epoch 5/32
16/16 - 1s - 43ms/step - accuracy: 0.6452 - loss: 2.6908
Epoch 6/32
16/16 - 1s - 43ms/step - accuracy: 0.6452 - loss: 2.6884
Epoch 7/32
16/16 - 1s - 44ms/step - accuracy: 0.6452 - loss: 2.6862
Epoch 8/32
16/16 - 1s - 43ms/step - accuracy: 0.6452 - loss: 2.6841
Epoch 9/32
16/16 - 1s - 44ms/step - accuracy: 0.6452 - loss: 2.6821
Epoch 10/32
16/16 - 1s - 44ms/step - accuracy: 0

ValueError: Input y contains NaN.

Epoch 17/88
13/13 - 1s - loss: 0.2686 - accuracy: 0.9031 - 760ms/epoch - 58ms/step
Epoch 18/88
13/13 - 1s - loss: 0.2763 - accuracy: 0.8929 - 745ms/epoch - 57ms/step
Epoch 19/88
13/13 - 1s - loss: 0.2030 - accuracy: 0.9294 - 756ms/epoch - 58ms/step
Epoch 20/88
13/13 - 1s - loss: 0.1853 - accuracy: 0.9301 - 740ms/epoch - 57ms/step
Epoch 21/88
13/13 - 1s - loss: 0.1833 - accuracy: 0.9342 - 740ms/epoch - 57ms/step
Epoch 22/88
13/13 - 1s - loss: 0.1674 - accuracy: 0.9388 - 751ms/epoch - 58ms/step
Epoch 23/88
13/13 - 1s - loss: 0.1683 - accuracy: 0.9391 - 740ms/epoch - 57ms/step
Epoch 24/88
13/13 - 1s - loss: 0.1634 - accuracy: 0.9379 - 770ms/epoch - 59ms/step
Epoch 25/88
13/13 - 1s - loss: 0.1690 - accuracy: 0.9373 - 762ms/epoch - 59ms/step
Epoch 26/88
13/13 - 1s - loss: 0.1507 - accuracy: 0.9444 - 755ms/epoch - 58ms/step
Epoch 27/88
13/13 - 1s - loss: 0.2110 - accuracy: 0.9164 - 742ms/epoch - 57ms/step
Epoch 28/88
13/13 - 1s - loss: 0.1455 - accuracy: 0.9432 - 750ms/epoch - 58ms/step
Epoc

Epoch 25/98
13/13 - 1s - loss: 1.7840 - accuracy: 0.1913 - 1s/epoch - 90ms/step
Epoch 26/98
13/13 - 1s - loss: 1.7840 - accuracy: 0.1913 - 1s/epoch - 89ms/step
Epoch 27/98
13/13 - 1s - loss: 1.7840 - accuracy: 0.1913 - 1s/epoch - 91ms/step
Epoch 28/98
13/13 - 1s - loss: 1.7840 - accuracy: 0.1913 - 1s/epoch - 90ms/step
Epoch 28: early stopping
46/46 [==============================] - 0s 4ms/step
Epoch 1/98
13/13 - 1s - loss: 8.9403 - accuracy: 0.1622 - 1s/epoch - 114ms/step
Epoch 2/98
13/13 - 1s - loss: 1.7869 - accuracy: 0.1913 - 1s/epoch - 90ms/step
Epoch 3/98
13/13 - 1s - loss: 1.7853 - accuracy: 0.1913 - 1s/epoch - 89ms/step
Epoch 4/98
13/13 - 1s - loss: 1.7846 - accuracy: 0.1913 - 1s/epoch - 88ms/step
Epoch 5/98
13/13 - 1s - loss: 1.7844 - accuracy: 0.1913 - 1s/epoch - 90ms/step
Epoch 6/98
13/13 - 1s - loss: 1.7842 - accuracy: 0.1913 - 1s/epoch - 90ms/step
Epoch 7/98
13/13 - 1s - loss: 1.7841 - accuracy: 0.1913 - 1s/epoch - 90ms/step
Epoch 8/98
13/13 - 1s - loss: 1.7841 - accuracy:

Epoch 21/98
13/13 - 1s - loss: 1.7840 - accuracy: 0.1914 - 1s/epoch - 93ms/step
Epoch 22/98
13/13 - 1s - loss: 1.7838 - accuracy: 0.1914 - 1s/epoch - 92ms/step
Epoch 22: early stopping
46/46 [==============================] - 0s 4ms/step
| 19        | 0.1914    | 2.496     | 466.9     | 0.1903    | 0.03816   | 98.04     | 1.039     | 1.918     | 1.543     | 0.1049    | 63.18     | 0.4924    | 4.245     |
Search took 64.84285565614701 minutes


In [148]:
optimum = nn_opt.max['params']
print(optimum)
print(round(optimum['optimizer']))

{'activation': 2.79884089544096, 'batch_size': 460.14665762139765, 'dropout': 0.7296061783380641, 'dropout_rate': 0.19126724140656393, 'epochs': 90.97701940610612, 'kernel': 1.9444298503238986, 'layers1': 1.2391884918766034, 'layers2': 2.42648957444599, 'learning_rate': 0.7631771981307285, 'neurons': 60.51494778125466, 'normalization': 0.770967179954561, 'optimizer': 3.456569174550735}
3


In [151]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']

optimum['activation'] = activationL[round(optimum['activation'])]
optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])
optimum['kernel'] = round(optimum['kernel'])
optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']

optimum['optimizer'] = optimizerL[round(optimum['optimizer'])]
optimum

{'activation': 'softsign',
 'batch_size': 460,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 91,
 'kernel': 2,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': 'Adadelta'}

## 04. Building CNN Model

In [195]:
## Building CNN Model
epochs = 91
batch_size = 460
#n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # There are 15 classes.
layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = 2
neurons = 61
normalization = 0.770967179954561
dropout = 0.7296061783380641
dropout_rate = 0.19126724140656393
optimizer = 'Adadelta'
learning_rate = 0.7631771981307285

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())
    
for i in range(layers1):
    model.add(Dense(neurons, activation=activation))
    
if dropout > 0.5:
    model.add(Dropout(dropout_rate, seed=123))
    
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))
    
model.add(MaxPooling1D())

model.add(Flatten())

model.add(Dense(n_classes, activation='softmax')) #softmax sigmoid

model.compile(loss='categorical_crossentropy', optimizer=Adadelta(learning_rate), metrics=['accuracy'])


In [193]:
#Turn y_train into one-hot format
y_train=utils.to_categorical(y_train, num_classes=15)


In [198]:
# Checking the shape of X_train and y_train
print(f"The shape of X_train:{X_train.shape}")
print(f"The shape of y_train: {y_train.shape}")

The shape of X_train:(18360, 15, 9)
The shape of y_train: (18360, 15)


In [197]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/91
40/40 - 1s - 26ms/step - accuracy: 0.5991 - loss: 1.4212
Epoch 2/91
40/40 - 1s - 15ms/step - accuracy: 0.6936 - loss: 0.9117
Epoch 3/91
40/40 - 1s - 15ms/step - accuracy: 0.7182 - loss: 0.8247
Epoch 4/91
40/40 - 1s - 15ms/step - accuracy: 0.7309 - loss: 0.7736
Epoch 5/91
40/40 - 1s - 15ms/step - accuracy: 0.7479 - loss: 0.7277
Epoch 6/91
40/40 - 1s - 15ms/step - accuracy: 0.7621 - loss: 0.6916
Epoch 7/91
40/40 - 1s - 14ms/step - accuracy: 0.7758 - loss: 0.6556
Epoch 8/91
40/40 - 1s - 14ms/step - accuracy: 0.7878 - loss: 0.6237
Epoch 9/91
40/40 - 1s - 15ms/step - accuracy: 0.7961 - loss: 0.5960
Epoch 10/91
40/40 - 1s - 14ms/step - accuracy: 0.8048 - loss: 0.5689
Epoch 11/91
40/40 - 1s - 15ms/step - accuracy: 0.8132 - loss: 0.5480
Epoch 12/91
40/40 - 1s - 15ms/step - accuracy: 0.8200 - loss: 0.5237
Epoch 13/91
40/40 - 1s - 15ms/step - accuracy: 0.8276 - loss: 0.5008
Epoch 14/91
40/40 - 1s - 15ms/step - accuracy: 0.8314 - loss: 0.4877
Epoch 15/91
40/40 - 1s - 15ms/step - accura

In [199]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([predictions[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([predictions[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [188]:
# Converting y_test to one-hot configuration
y_test = to_categorical(y_test, num_classes=15)

In [200]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        2855        49         5       1           0         1       0   
BELGRADE       47       825         5       0           0         1       1   
BUDAPEST       11        18       145       1           0         1       0   
DEBILT          2         5        14      55           1         1       0   
DUSSELDORF      5         0         0       5          15         7       0   
HEATHROW        3         0         0       0           4        53       0   
KASSEL          0         0         2       1           0         0       4   
LJUBLJANA       3         1         0       0           0         0       0   
MAASTRICHT      1         0         1       0           2         0       0   
MADRID         22         0         3       0           0         3       0   
MUNCHENB 